In [ ]:
# imported from code along lesson
import random
suits = ("Hearts", "Diamonds", "Spades", "Clubs")
ranks = ("Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Jack", "Queen", "King", "Ace")
values = {"Two": 2, "Three": 3, "Four": 4, "Five": 5, "Six": 6, "Seven": 7, "Eight": 8,
          "Nine": 9, "Ten": 10, "Jack": 10, "Queen": 10, "King": 10, "Ace": 0}
dealer_choice = (1, 11)

In [ ]:
# imported from code along lesson
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return self.rank + " of " + self.suit

In [ ]:
# imported from code along lesson
class Deck:
    
    def __init__(self):
        
        self.all_cards = []
        
        for suit in suits:
            
            for rank in ranks:
                
                card = Card(suit, rank)
                self.all_cards.append(card)
                
    def shuffle(self):
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        return self.all_cards.pop()

In [ ]:
class Player:
    
    def __init__(self, name):
        
        self.name = name
        self.hand = []
        self.balance = 1000
        self.card_sum = 0
        self.bust = False
        
    def hit(self, new):
        self.hand.append(new)
        
    def win(self, winnings):
        self.balance += winnings
        
    def loss(self, loss):
        self.balance -= loss
        
    def reset(self):
        self.hand = []
        self.bust = False
        
    def check_hand_sum(self):
        self.card_sum = 0
        for x in range(len(self.hand)):
            self.card_sum += self.hand[x].value
    
    def __str__(self):
        return f"Your current balance is {self.balance}"

In [ ]:
class Dealer:
    
    def __init__(self):
        
        self.hand = []
        self.card_sum = 0
        self.bust = False
        
    def reset(self):
        self.hand = []
        self.bust = False
        
    def hit(self, new):
        self.hand.append(new)
        
    def check_hand_sum(self):
        self.card_sum = 0
        for x in range(len(self.hand)):
            self.card_sum += self.hand[x].value

In [ ]:
# game setup

new_deck = Deck()
new_deck.shuffle()

p1 = input("Please enter your name: ")

In [ ]:
player = Player(p1)
dealer = Dealer()

In [ ]:
while player.balance > 0 and len(new_deck.all_cards) > 0:
    
    print(player)
    bet = input("How much would you like to bet this round (Enter 0 to stop): ")

    while bet.isnumeric() == False:

        print("Error, that is not a valid number!")
        bet = input("How much would you like to bet this round: ")

    while int(bet) > player.balance:

        print("Error, you cannot bet more than your current balance!")
        print(player)
        bet = input("How much would you like to bet this round: ")
    
    bet = int(bet)
    
    if bet == 0:
        break
    
    for x in range(2):
        player.hit(new_deck.deal_one())
        dealer.hit(new_deck.deal_one())
    
    print("Dealer's hand:")
    print(f"{dealer.hand[0].rank} of {dealer.hand[0].suit}. Value of: {dealer.hand[0].value}")
    print("A face down card")
    dealer.check_hand_sum()
    
    
    # -------------
    # Player's turn
    # -------------
    
    print("Your hand:")
    for x in range(len(player.hand)):
        print(f"{player.hand[x].rank} of {player.hand[x].suit}. Value of: {player.hand[x].value}")
        
        if player.hand[x].value == 0:
            choice = input("You have an ace! Choose it's value (1 or 11): ")
            
            while choice not in ["1", "11"]:
                choice = input("Wrong value! Try again (1 or 11): ")
            
            player.hand[x].value = int(choice)
    
    player.check_hand_sum()
    
    if player.card_sum == 21 and len(player.hand) == 2:
            print("Blackjack! You win!")
            bet = bet * 1.5
            player.win(bet)
            print(player)
            continue
    
    player_turn = True
    
    print("--------------")
    print("Player's turn")
    print("--------------")
    
    while player_turn and not player.bust:
        
        print("Your hand currently:")
        for x in range(len(player.hand)):
            print(f"{player.hand[x].rank} of {player.hand[x].suit}. Value of: {player.hand[x].value}")

            if player.hand[x].value == 0:
                choice = input("You have an ace! Choose it's value (1 or 11): ")

                while choice not in ["1", "11"]:
                    choice = input("Wrong value! Try again (1 or 11): ")

                player.hand[x].value = int(choice)
        
        player.check_hand_sum()
        
        if player.card_sum > 21:
            player.bust = True
            print("You have busted!")
            break
        
        player_move = input("Hit or stay?: ")
        
        while player_move not in ["Hit", "Stay", "hit", "stay"]:
            player_move = input("Error! Please try again, hit or stay: ")
        
        if player_move in ["Stay", "stay"]:
            player_turn = False
        
        else:
            player.hit(new_deck.deal_one())
        
    # -------------
    # Dealer's turn
    # -------------
    
    print("--------------")
    print("Dealer's turn")
    print("--------------")
    
    if not player.bust:
        
        while dealer.card_sum <= player.card_sum and not dealer.bust:

            for x in range(len(dealer.hand)):

                if dealer.hand[x].value == 0:

                    print("Dealer has an ace! It will choose randomly between 1 and 11")
                    dealer.hand[x].value = random.choice(dealer_choice)


            dealer.check_hand_sum()

            if dealer.card_sum == 21:
                break

            dealer.hit(new_deck.deal_one())
            dealer.check_hand_sum()
            
            if dealer.card_sum > 21:
                print("BUSTING CHECK")
                dealer.bust = True
                print("Dealer has busted!")
                break
        
    # -------------
    # Detect winner
    # -------------
    
    if not player.bust and not dealer.bust:
        print("VERIFYING WINNER")
        
        if player.card_sum > dealer.card_sum:
            print(f"{player.name} wins! Your score: {player.card_sum}, dealer's score {dealer.card_sum}")
            player.win(bet)
            
        elif dealer.card_sum > player.card_sum:
            print(f"Dealer wins! Dealer's score: {dealer.card_sum}, your score: {player.card_sum}")
            player.loss(bet)
            
        elif player.card_sum == dealer.card_sum:
            print(f"Tie game! Both you and the dealer have a score of {player.card_sum}")
    
    elif not player.bust and dealer.bust:
        print(f"{player.name} wins! Your score: {player.card_sum}, dealer busted with {dealer.card_sum}!")
        player.win(bet)
    
    elif not dealer.bust and player.bust:
        print(f"Dealer wins! Dealer's score: {dealer.card_sum}, you busted with {player.card_sum}!")
        player.loss(bet)
    
    player.reset()
    dealer.reset()
    
    print("--------------------")
    print("New round!")
    print("--------------------")

print("===============================")
if player.balance == 0:
    print("Game over! You are out of money")
else:
    print("Game over!")